In [1]:
from ugot import ugot
from IPython.display import clear_output
import time
got = ugot.UGOT()
got.initialize("192.168.1.164")

192.168.1.164:50051


In [ ]:
# Arm control

# Open gripper
got.mechanical_clamp_release()

# Close gripper
got.mechanical_clamp_close()

# Reset the arm
got.mechanical_arms_restory()

# Joint control
# Angle 1 = closest to body, controls left and right
# Angle 2 & 3 = middle and furthest, controls up and down
# Duration is in milliseconds
got.mechanical_joint_control(angle1=0, angle2=-45, angle3=-45, duration=500)
# If you want to delay, need to add time.sleep()

In [ ]:
# Go down with a certain duration, and delay
def down(delay, duration):
    got.mechanical_joint_control(angle1=0, angle2=-45, angle3=-45, duration=duration*1000)

    if delay > 0:
        time.sleep(delay)

# Go up
def up(delay, duration):
    got.mechanical_joint_control(angle1=0, angle2=45, angle3=45, duration=duration*1000)

    if delay > 0:
        time.sleep(delay)

down(delay=0.5, duration=0.5)

In [ ]:
# Throwing UNDERHAND
got.mechanical_clamp_release()
down(delay=0.5, duration=0.5)
time.sleep(1)
got.mechanical_clamp_close()
time.sleep(1)
up(delay=0, duration=0.0100) # Fast up swing!
got.mechanical_clamp_release()

In [ ]:
got.load_models(["apriltag_qrcode"])

THRESHOLD_DISTANCE = 0.10

try:
    while True:
        tag_info = got.get_apriltag_total_info()

        if tag_info:
            # Center x coordinate
            c_x = tag_info[0][1]
            # Distance measurement
            dist = tag_info[0][6]

            if dist > THRESHOLD_DISTANCE:
                if c_x > 360:
                    got.mecanum_translate_speed(angle=45, speed=20)
                elif c_x < 280:
                    got.mecanum_translate_speed(angle=-45, speed=20)
                else:
                    got.mecanum_translate_speed(angle=0, speed=20)
            
            else:
                got.mecanum_stop()
                print("Reached!")
                got.mechanical_clamp_release()
                down(delay=0.5, duration=0.5)
                got.mechanical_clamp_close()
                time.sleep(1)
                up(delay=0.5, duration=0.5)



except KeyboardInterrupt:
    got.mecanum_stop()
    print("Done!")